In [1]:
import pandas as pd

train_data_file = "./zhengqi_train.txt"
test_data_file =  "./zhengqi_test.txt"

train_data = pd.read_csv(train_data_file, sep='\t', encoding='utf-8')
test_data = pd.read_csv(test_data_file, sep='\t', encoding='utf-8')

In [2]:
epsilon=1e-5

#组交叉特征，可以自行定义，如增加： x*x/y, log(x)/y 等等
func_dict = {
            'add': lambda x,y: x+y,
            'mins': lambda x,y: x-y,
            'div': lambda x,y: x/(y+epsilon),
            'multi': lambda x,y: x*y
            }

In [3]:
def auto_features_make(train_data,test_data,func_dict,col_list):
    train_data, test_data = train_data.copy(), test_data.copy()
    for col_i in col_list:
        for col_j in col_list:
            for func_name, func in func_dict.items():
                for data in [train_data,test_data]:
                    func_features = func(data[col_i],data[col_j])
                    col_func_features = '-'.join([col_i,func_name,col_j])
                    data[col_func_features] = func_features
    return train_data,test_data

In [4]:
train_data2, test_data2 = auto_features_make(train_data,test_data,func_dict,col_list=test_data.columns)

In [5]:
from sklearn.decomposition import PCA   #主成分分析法

#PCA方法降维
pca = PCA(n_components=500)
train_data2_pca = pca.fit_transform(train_data2.iloc[:,0:-1])
test_data2_pca = pca.transform(test_data2)
train_data2_pca = pd.DataFrame(train_data2_pca)
test_data2_pca = pd.DataFrame(test_data2_pca)
train_data2_pca['target'] = train_data2['target']

In [6]:
X_train2 = train_data2[test_data2.columns].values
y_train = train_data2['target']

In [ ]:
# ls_validation i
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import numpy as np

# 5折交叉验证
Folds=5
kf = KFold(len(X_train2), n_splits=Folds, random_state=2019, shuffle=True)
# 记录训练和预测MSE
MSE_DICT = {
    'train_mse':[],
    'test_mse':[]
}

# 线下训练预测
for i, (train_index, test_index) in enumerate(kf.split(X_train2)):
    # lgb树模型
    lgb_reg = lgb.LGBMRegressor(
        learning_rate=0.01,
        max_depth=-1,
        n_estimators=5000,
        boosting_type='gbdt',
        random_state=2019,
        objective='regression',
    )
   
    # 切分训练集和预测集
    X_train_KFold, X_test_KFold = X_train2[train_index], X_train2[test_index]
    y_train_KFold, y_test_KFold = y_train[train_index], y_train[test_index]
    
    # 训练模型
    lgb_reg.fit(
            X=X_train_KFold,y=y_train_KFold,
            eval_set=[(X_train_KFold, y_train_KFold),(X_test_KFold, y_test_KFold)],
            eval_names=['Train','Test'],
            early_stopping_rounds=100,
            eval_metric='MSE',
            verbose=50
        )


    # 训练集预测 测试集预测
    y_train_KFold_predict = lgb_reg.predict(X_train_KFold,num_iteration=lgb_reg.best_iteration_)
    y_test_KFold_predict = lgb_reg.predict(X_test_KFold,num_iteration=lgb_reg.best_iteration_) 
    
    print('第{}折 训练和预测 训练MSE 预测MSE'.format(i))
    train_mse = mean_squared_error(y_train_KFold_predict, y_train_KFold)
    print('------\n', '训练MSE\n', train_mse, '\n------')
    test_mse = mean_squared_error(y_test_KFold_predict, y_test_KFold)
    print('------\n', '预测MSE\n', test_mse, '\n------\n')
    
    MSE_DICT['train_mse'].append(train_mse)
    MSE_DICT['test_mse'].append(test_mse)
print('------\n', '训练MSE\n', MSE_DICT['train_mse'], '\n', np.mean(MSE_DICT['train_mse']), '\n------')
print('------\n', '预测MSE\n', MSE_DICT['test_mse'], '\n', np.mean(MSE_DICT['test_mse']), '\n------')

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.418978	Test's l2: 0.106374
[100]	Train's l2: 0.203693	Test's l2: 0.02276
[150]	Train's l2: 0.114486	Test's l2: 0.00527795
[200]	Train's l2: 0.0741934	Test's l2: 5.99266e-05
[250]	Train's l2: 0.0535396	Test's l2: 0.00036171
[300]	Train's l2: 0.041529	Test's l2: 0.00267813
Early stopping, best iteration is:
[221]	Train's l2: 0.0640274	Test's l2: 6.95547e-08
第0折 训练和预测 训练MSE 预测MSE
------
 训练MSE
 0.0640273654375399 
------
------
 预测MSE
 6.954770450692039e-08 
------

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.419128	Test's l2: 0.142103
[100]	Train's l2: 0.203838	Test's l2: 0.0997735
[150]	Train's l2: 0.114537	Test's l2: 0.0765469
[200]	Train's l2: 0.0742645	Test's l2: 0.0612271
[250]	Train's l2: 0.0536164	Test's l2: 0.0471729
[300]	Train's l2: 0.0415117	Test's l2: 0.0427081
[350]	Train's l2: 0.0333767	Test's l2: 0.0399094
[400]	Train's l2: 0.0275956	Test's l2: 0.0375303
[4

[150]	Train's l2: 0.114559	Test's l2: 0.0172525
[200]	Train's l2: 0.0742264	Test's l2: 0.00504282
[250]	Train's l2: 0.0535831	Test's l2: 0.00153378
[300]	Train's l2: 0.0414425	Test's l2: 7.48706e-06
[350]	Train's l2: 0.0333508	Test's l2: 0.000496099
[400]	Train's l2: 0.0275969	Test's l2: 0.00111861
Early stopping, best iteration is:
[305]	Train's l2: 0.040494	Test's l2: 3.75824e-08
第12折 训练和预测 训练MSE 预测MSE
------
 训练MSE
 0.04049397966649528 
------
------
 预测MSE
 3.758870105458577e-08 
------

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.419187	Test's l2: 0.016239
[100]	Train's l2: 0.203862	Test's l2: 0.0128048
[150]	Train's l2: 0.114614	Test's l2: 0.0104147
[200]	Train's l2: 0.0743045	Test's l2: 0.00526246
[250]	Train's l2: 0.0536267	Test's l2: 0.00508631
[300]	Train's l2: 0.0415942	Test's l2: 0.00615596
Early stopping, best iteration is:
[219]	Train's l2: 0.0649886	Test's l2: 0.00402294
第13折 训练和预测 训练MSE 预测MSE
------
 训练MSE
 0.06498858566375795 
-----

[850]	Train's l2: 0.00689111	Test's l2: 0.000434364
[900]	Train's l2: 0.00603225	Test's l2: 0.000305302
[950]	Train's l2: 0.00529106	Test's l2: 0.000337115
[1000]	Train's l2: 0.00465794	Test's l2: 0.000242977
[1050]	Train's l2: 0.00409975	Test's l2: 0.000257165
Early stopping, best iteration is:
[985]	Train's l2: 0.00483596	Test's l2: 0.000207193
第20折 训练和预测 训练MSE 预测MSE
------
 训练MSE
 0.004835961628789988 
------
------
 预测MSE
 0.00020719455433872638 
------

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.419024	Test's l2: 0.00281614
[100]	Train's l2: 0.203791	Test's l2: 0.00726691
Early stopping, best iteration is:
[19]	Train's l2: 0.696433	Test's l2: 0.000226693
第21折 训练和预测 训练MSE 预测MSE
------
 训练MSE
 0.6964330189928877 
------
------
 预测MSE
 0.00022669317466322212 
------

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.41894	Test's l2: 0.130452
[100]	Train's l2: 0.203652	Test's l2: 0.0556921
[150]	Train's l2: 0.114451	

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.418976	Test's l2: 0.00670833
[100]	Train's l2: 0.203692	Test's l2: 0.0703949
Early stopping, best iteration is:
[34]	Train's l2: 0.54186	Test's l2: 4.31568e-09
第34折 训练和预测 训练MSE 预测MSE
------
 训练MSE
 0.5418596769252172 
------
------
 预测MSE
 4.3150200220118765e-09 
------

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.41898	Test's l2: 0.547579
[100]	Train's l2: 0.203807	Test's l2: 0.485001
[150]	Train's l2: 0.114558	Test's l2: 0.456115
[200]	Train's l2: 0.0742933	Test's l2: 0.426775
[250]	Train's l2: 0.0536172	Test's l2: 0.409048
[300]	Train's l2: 0.0415185	Test's l2: 0.378165
[350]	Train's l2: 0.0334627	Test's l2: 0.371452
[400]	Train's l2: 0.0276288	Test's l2: 0.355203
[450]	Train's l2: 0.023069	Test's l2: 0.334513
[500]	Train's l2: 0.0194404	Test's l2: 0.337011
[550]	Train's l2: 0.0165069	Test's l2: 0.342605
Early stopping, best iteration is:
[477]	Train's l2: 0.0210082	

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.419116	Test's l2: 0.0139555
[100]	Train's l2: 0.203761	Test's l2: 0.0200994
Early stopping, best iteration is:
[1]	Train's l2: 0.95122	Test's l2: 0.00524442
第48折 训练和预测 训练MSE 预测MSE
------
 训练MSE
 0.9512195532986107 
------
------
 预测MSE
 0.005244422362982837 
------

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.419094	Test's l2: 0.0355316
[100]	Train's l2: 0.203786	Test's l2: 0.00930918
[150]	Train's l2: 0.114585	Test's l2: 0.00445071
[200]	Train's l2: 0.0742982	Test's l2: 0.00220223
[250]	Train's l2: 0.0535865	Test's l2: 0.000789025
[300]	Train's l2: 0.0415061	Test's l2: 0.000862087
[350]	Train's l2: 0.0333907	Test's l2: 0.000590142
[400]	Train's l2: 0.0275241	Test's l2: 0.00101836
[450]	Train's l2: 0.0230942	Test's l2: 0.000421242
[500]	Train's l2: 0.0194392	Test's l2: 0.000493269
[550]	Train's l2: 0.0165173	Test's l2: 0.000470089
Early stopping, best iteration is:
[454

...

第180折 训练和预测 训练MSE 预测MSE
------
 训练MSE
 0.036991782190078364 
------
------
 预测MSE
 0.014795776705694055 
------

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.418765	Test's l2: 0.727971
[100]	Train's l2: 0.203644	Test's l2: 0.203594
[150]	Train's l2: 0.114631	Test's l2: 0.0536387
[200]	Train's l2: 0.0744115	Test's l2: 0.0146741
[250]	Train's l2: 0.0537382	Test's l2: 0.000970886
[300]	Train's l2: 0.0416628	Test's l2: 0.0010149
[350]	Train's l2: 0.0334735	Test's l2: 0.00361907
Early stopping, best iteration is:
[268]	Train's l2: 0.0487208	Test's l2: 2.10364e-07
第181折 训练和预测 训练MSE 预测MSE
------
 训练MSE
 0.04872076519212383 
------
------
 预测MSE
 2.10393110536295e-07 
------

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.418989	Test's l2: 0.536342
[100]	Train's l2: 0.203791	Test's l2: 0.500751
Early stopping, best iteration is:
[1]	Train's l2: 0.951066	Test's l2: 0.493265
第182折 训练和预测 训练MSE 预测MSE
------
 训练MSE
 0.95106552426

Early stopping, best iteration is:
[58]	Train's l2: 0.369996	Test's l2: 0.158263
第195折 训练和预测 训练MSE 预测MSE
------
 训练MSE
 0.3699955791686925 
------
------
 预测MSE
 0.15826312640458212 
------

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.41861	Test's l2: 1.48407
[100]	Train's l2: 0.203567	Test's l2: 0.870214
[150]	Train's l2: 0.114457	Test's l2: 0.536823
[200]	Train's l2: 0.074181	Test's l2: 0.448233
[250]	Train's l2: 0.0534517	Test's l2: 0.390963
[300]	Train's l2: 0.0414248	Test's l2: 0.342592
[350]	Train's l2: 0.0333144	Test's l2: 0.324272
[400]	Train's l2: 0.0274787	Test's l2: 0.328511
Early stopping, best iteration is:
[345]	Train's l2: 0.0340093	Test's l2: 0.320523
第196折 训练和预测 训练MSE 预测MSE
------
 训练MSE
 0.034009317717330595 
------
------
 预测MSE
 0.3205228603363154 
------

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.418908	Test's l2: 0.381869
[100]	Train's l2: 0.203621	Test's l2: 0.118174
[150]	Train's l2: 0.1

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.419103	Test's l2: 0.0194681
[100]	Train's l2: 0.203753	Test's l2: 0.015892
[150]	Train's l2: 0.114496	Test's l2: 0.00927958
[200]	Train's l2: 0.0741036	Test's l2: 0.0106643
[250]	Train's l2: 0.0534423	Test's l2: 0.00540217
[300]	Train's l2: 0.0412979	Test's l2: 0.00306
[350]	Train's l2: 0.0332096	Test's l2: 0.00216028
[400]	Train's l2: 0.02745	Test's l2: 0.0023989
[450]	Train's l2: 0.0229696	Test's l2: 0.00258294
[500]	Train's l2: 0.019331	Test's l2: 0.00278702
[550]	Train's l2: 0.016423	Test's l2: 0.00312754
Early stopping, best iteration is:
[477]	Train's l2: 0.0208962	Test's l2: 0.0019671
第206折 训练和预测 训练MSE 预测MSE
------
 训练MSE
 0.020896203014593984 
------
------
 预测MSE
 0.001967095619460403 
------

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.419073	Test's l2: 0.0957326
[100]	Train's l2: 0.203831	Test's l2: 0.0463486
[150]	Train's l2: 0.114559	Test's l2: 0.0319054
[2

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.418985	Test's l2: 0.0126732
[100]	Train's l2: 0.20374	Test's l2: 0.000837039
[150]	Train's l2: 0.114525	Test's l2: 0.0075325
Early stopping, best iteration is:
[85]	Train's l2: 0.249293	Test's l2: 2.33816e-07
第218折 训练和预测 训练MSE 预测MSE
------
 训练MSE
 0.24929301802127538 
------
------
 预测MSE
 2.338040218148175e-07 
------

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.419018	Test's l2: 0.00054657
[100]	Train's l2: 0.203821	Test's l2: 0.000439907
Early stopping, best iteration is:
[1]	Train's l2: 0.951229	Test's l2: 1.40832e-06
第219折 训练和预测 训练MSE 预测MSE
------
 训练MSE
 0.9512292802323905 
------
------
 预测MSE
 1.4083184222840587e-06 
------

Training until validation scores don't improve for 100 rounds
[50]	Train's l2: 0.41914	Test's l2: 0.00046269
[100]	Train's l2: 0.203815	Test's l2: 0.017747
[150]	Train's l2: 0.114548	Test's l2: 0.0532437
Early stopping, best iteration is:
[5

[2450]	Train's l2: 0.00024449	Test's l2: 0.00242461
[2500]	Train's l2: 0.000226238	Test's l2: 0.00239461
[2550]	Train's l2: 0.000209565	Test's l2: 0.00240033
[2600]	Train's l2: 0.000194168	Test's l2: 0.0023347
[2650]	Train's l2: 0.000180318	Test's l2: 0.00232796
[2700]	Train's l2: 0.000167814	Test's l2: 0.00228473
[2750]	Train's l2: 0.000156238	Test's l2: 0.00227587
[2800]	Train's l2: 0.000145601	Test's l2: 0.00221113
[2850]	Train's l2: 0.000136047	Test's l2: 0.00220042
[2900]	Train's l2: 0.000127254	Test's l2: 0.00216278
[2950]	Train's l2: 0.000119191	Test's l2: 0.00213557
[3000]	Train's l2: 0.000111702	Test's l2: 0.00212836
[3050]	Train's l2: 0.000104858	Test's l2: 0.00210898
[3100]	Train's l2: 9.85169e-05	Test's l2: 0.00209333
[3150]	Train's l2: 9.26579e-05	Test's l2: 0.00206293
[3200]	Train's l2: 8.7301e-05	Test's l2: 0.00204348
[3250]	Train's l2: 8.22032e-05	Test's l2: 0.00205177
[3300]	Train's l2: 7.74639e-05	Test's l2: 0.00204315
[3350]	Train's l2: 7.30718e-05	Test's l2: 0.00203